In [5]:
import sys
print(sys.executable)

c:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\.venv\Scripts\python.exe


In [6]:
import pandas as pd

df_sub = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\US Electric Power Transmission Substations.csv")
df_iou = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\iou_zipcodes_U.S._Electric_Companies_and_Rates.csv")
df_non = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\non_iou_zipcodes_U.S._Electric_Companies_Rates_and_Zipcode (2020).csv")

print(df_sub.shape, df_iou.shape, df_non.shape)



C:\Users\palmi\AppData\Local\Temp\ipykernel_15472\3807714741.py:3: DtypeWarning: Columns (7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sub = pd.read_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\raw\US Electric Power Transmission Substations.csv")


(79687, 26) (52177, 9) (28027, 9)


UNIR IOU Y NON-IOU Y MARCAR TIPO

In [7]:
df_iou["utility_type"] = "IOU"
df_non["utility_type"] = "NON_IOU"

df_utils = pd.concat([df_iou, df_non], ignore_index=True)
print("Utilities unidas:", df_utils.shape)
df_utils.head()


Utilities unidas: (80204, 10)


,zip,eiaid,utility_name,state,service_type,ownership,comm_rate,ind_rate,res_rate,utility_type
0,85321,176,Ajo Improvement Co,AZ,Bundled,Investor Owned,0.087890,0.000000,0.093887,IOU
1,36560,195,Alabama Power Co,AL,Bundled,Investor Owned,0.121895,0.063652,0.135057,IOU
2,36513,195,Alabama Power Co,AL,Bundled,Investor Owned,0.121895,0.063652,0.135057,IOU
3,36280,195,Alabama Power Co,AL,Bundled,Investor Owned,0.121895,0.063652,0.135057,IOU
4,35473,195,Alabama Power Co,AL,Bundled,Investor Owned,0.121895,0.063652,0.135057,IOU


ENCONTRAR LAS COLUMNAS DE STATE Y ZIP

In [8]:
state_cols = [c for c in df_utils.columns if "state" in c.lower()]
zip_cols = [c for c in df_utils.columns if "zip" in c.lower()]

print("State cols:", state_cols)
print("Zip cols:", zip_cols)


State cols: ['state']
Zip cols: ['zip']


FILTRAR TEXAS

In [9]:
state_col = state_cols[0]  
df_tx = df_utils[df_utils[state_col].astype(str).str.upper().str.strip() == "TX"].copy()

print("Utilities Texas:", df_tx.shape)
df_tx.head()


Utilities Texas: (3085, 10)


,zip,eiaid,utility_name,state,service_type,ownership,comm_rate,ind_rate,res_rate,utility_type
12841,79932,5701,El Paso Electric Co,TX,Bundled,Investor Owned,0.08918,0.046611,0.120553,IOU
12842,79903,5701,El Paso Electric Co,TX,Bundled,Investor Owned,0.08918,0.046611,0.120553,IOU
12843,79906,5701,El Paso Electric Co,TX,Bundled,Investor Owned,0.08918,0.046611,0.120553,IOU
12844,79901,5701,El Paso Electric Co,TX,Bundled,Investor Owned,0.08918,0.046611,0.120553,IOU
12845,79851,5701,El Paso Electric Co,TX,Bundled,Investor Owned,0.08918,0.046611,0.120553,IOU


NORMALIZAR ZIP PARA QUE QUEDE SIEMPRE DE 5 DIGITOS

In [10]:
zip_col = zip_cols[0]

df_tx[zip_col] = (
    df_tx[zip_col]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
    .str.zfill(5)
)

df_tx[[zip_col]].head()


,zip
12841,79932
12842,79903
12843,79906
12844,79901
12845,79851


SUBESTACIONES: VER SI TIENE COLUMNA ESTADO Y FILTRAR TX SI SE PUEDE

In [11]:
sub_state_cols = [c for c in df_sub.columns if "state" in c.lower()]
print("State cols en subestaciones:", sub_state_cols)


State cols en subestaciones: ['STATE']


In [12]:
if sub_state_cols:
    sub_state_col = sub_state_cols[0]
    df_sub_tx = df_sub[df_sub[sub_state_col].astype(str).str.upper().str.strip() == "TX"].copy()
else:
    df_sub_tx = df_sub.copy()

print("Substations (TX o full):", df_sub_tx.shape)
df_sub_tx.head()


Substations (TX o full): (5314, 26)


,X,Y,OBJECTID,ID,NAME,CITY,STATE,ZIP,TYPE,STATUS,...,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,LINES,MAX_VOLT,MIN_VOLT,MAX_INFER,MIN_INFER
41643,-1.047346e+07,3.964264e+06,41644,156744,UNKNOWN156744,TEXARKANA,TX,75503,SUBSTATION,IN SERVICE,...,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2016/09/16 00:00:00+00,IMAGERY,2016/09/16 00:00:00+00,2,69.0,69.0,N,N
46783,-1.046831e+07,3.758031e+06,46784,161924,TAP161924,JOAQUIN,TX,75954,TAP,IN SERVICE,...,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2019/03/26 00:00:00+00,IMAGERY,2019/03/26 00:00:00+00,2,138.0,138.0,N,N
46784,-1.046828e+07,3.757988e+06,46785,161925,UNKNOWN161925,JOAQUIN,TX,75954,SUBSTATION,IN SERVICE,...,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2019/03/26 00:00:00+00,IMAGERY,2019/03/26 00:00:00+00,0,-999999.0,-999999.0,Y,Y
48499,-1.115529e+07,4.144043e+06,48500,166905,UNKNOWN166905,WELLINGTON,TX,79095,SUBSTATION,NOT AVAILABLE,...,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2019/07/03 00:00:00+00,IMAGERY,2019/07/03 00:00:00+00,0,-999999.0,-999999.0,N,N
54089,-1.049590e+07,3.784288e+06,54090,172446,TAP172446,CARTHAGE,TX,75633,TAP,NOT AVAILABLE,...,ELECTRIC BULK POWER TRANSMISSION AND CONTROL,IMAGERY,2020/04/13 00:00:00+00,IMAGERY,2020/04/13 00:00:00+00,2,-999999.0,-999999.0,N,N


GUARDAR ARCHIVOS PROCESADOS LISTOS PARA EDA

In [13]:
import os
os.makedirs(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\processed", exist_ok=True)

df_tx.drop_duplicates().to_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\processed\utilities_tx.csv", index=False)
df_sub_tx.drop_duplicates().to_csv(r"C:\Users\palmi\OneDrive\Escritorio\Bootcamp\texas-utility-capstone\data\processed\substations_tx_or_clean.csv", index=False)

print("✅ Guardado utilities_tx.csv y substations_tx_or_clean.csv en data/processed")


✅ Guardado utilities_tx.csv y substations_tx_or_clean.csv en data/processed


In [14]:
print("Utilities TX:", df_tx.shape)
print("Substations:", df_sub_tx.shape)
print(df_tx["utility_type"].value_counts())


Utilities TX: (3085, 10)
Substations: (5314, 26)
utility_type
NON_IOU    2389
IOU         696
Name: count, dtype: int64
